In [1]:
import tensorflow as tf

2024-04-08 18:27:29.381430: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-08 18:27:34.240348: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from tensorflow import keras as keras
from keras import layers as layers

In [3]:
import os, timeit
from skimage.filters import threshold_otsu
import numpy as np
from math import inf as inf

In [4]:
import pandas as pd

In [5]:
from spectral.io import envi as envi
from spectral import imshow

In [6]:
from sklearn.decomposition import IncrementalPCA

In [7]:
import sys

In [8]:
import random

tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

In [9]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(physical_devices[1], 'GPU')

Num GPUs Available:  2


In [10]:
from sys import platform
DATA_DIRECTORY = ""
SLASH = ""
if platform == "linux" or platform == "linux2":
    DATA_DIRECTORY = "/home/tyagi/Desktop/wheat/data/BULK/"
    SLASH = "/"
elif platform == "win32":
    DATA_DIRECTORY = "D:\wheat\data\BULK\\"
    SLASH="\\"

In [11]:
#Constants
BAND_NUMBER = 60
FILLED_AREA_RATIO = 0.9
TRAIN_IMAGE_COUNT = 1200
VAL_IMAGE_COUNT = 400
TEST_IMAGE_COUNT = 400
NUM_VARIETIES = 4

IMAGE_WIDTH = 30
IMAGE_HEIGHT = 30

In [12]:
ACTIVATION_TYPE =  "relu"
BATCH_SIZE = 2*NUM_VARIETIES
LEARNING_RATE_BASE = 0.0001

In [13]:
from enum import Enum

class filter_method(Enum):
    none = 0
    snv = 1
    msc = 2
    savgol = 3
    
FILTER = filter_method(2).name

# to be set if filter chosen is savgol
WINDOW = 7
ORDER = 2
DERIVATIVE = "none"

In [14]:
from enum import Enum
 
class feature_extraction_method(Enum):
    none = 0
    pca_loading = 1
    lda = 2
    ipca = 3

FEATURE_EXTRACTION = feature_extraction_method(0).name

NUM_OF_BANDS = 3
if FEATURE_EXTRACTION == "pca_loading" or FEATURE_EXTRACTION == "ipca":
    NUM_OF_BANDS = 8
elif FEATURE_EXTRACTION == "lda":
    NUM_OF_BANDS = 3
    assert NUM_OF_BANDS <= min(NUM_VARIETIES-1,168),"NUM_OF_BANDS is greater."


REMOVE_NOISY_BANDS = False
FIRST_BAND = 15
LAST_BAND = 161

In [15]:
def start_timer():
    print("Testing started")
    return timeit.default_timer()

def end_timer():
    return timeit.default_timer()

def show_time(tic,toc): 
    test_time = toc - tic
    print('Testing time (s) = ' + str(test_time) + '\n')

In [16]:
# List for All varieties
VARIETIES = []
VARIETIES_CODE = {}

for name in os.listdir(DATA_DIRECTORY):
    if (name.endswith(".hdr") or name.endswith(".bil")):
        continue
    VARIETIES_CODE[name] = len(VARIETIES)
    VARIETIES.append(name)
    if len(VARIETIES)==NUM_VARIETIES:
        break

In [17]:
def dataset_file_name(variety):
    name = "./dataset/V"+str(variety).zfill(3)+"_FilledArea_"+str(FILLED_AREA_RATIO)+"_NumOfBands_"+str(NUM_OF_BANDS)+"_FB_"+str(FIRST_BAND)+"_LB_"+str(LAST_BAND)+"_BandNo_"+str(BAND_NUMBER)+"_ImageHeight_"+str(IMAGE_HEIGHT)+"_ImageWidth_"+str(IMAGE_WIDTH)+"_FILTER_"+str(FILTER)+"_FeatureExtraction_"+str(FEATURE_EXTRACTION)
    if REMOVE_NOISY_BANDS:
        name+="_REMOVE_NOISY_BANDS_"+str(REMOVE_NOISY_BANDS)
    if FILTER == "savgol":
        name+="_WINDOW_"+str(WINDOW)+"_ORDER_"+str(ORDER)
    return name

In [18]:
x_train = []
y_train = []
x_val = []
y_val = []
test_dataset=[]
test_dataset_label = []

for idx, v in enumerate(VARIETIES):
    print("idx: ",idx)
    if idx >= NUM_VARIETIES:
        break
    x_train= x_train + np.load(dataset_file_name(v)+"_train_dataset.npy").tolist()
    y_train = y_train + np.load(dataset_file_name(v)+"_train_dataset_label.npy").tolist()
    x_val= x_val + np.load(dataset_file_name(v)+"_val_dataset.npy").tolist()
    y_val = y_val + np.load(dataset_file_name(v)+"_val_dataset_label.npy").tolist()
    test_dataset = test_dataset + np.load(dataset_file_name(v)+"_test_dataset.npy").tolist()
    test_dataset_label = test_dataset_label + np.load(dataset_file_name(v)+"_test_dataset_label.npy").tolist()
    
x_train = np.array(x_train)
y_train = np.array(y_train)
x_val = np.array(x_val)
y_val = np.array(y_val)
test_dataset = np.array(test_dataset)
test_dataset_label = np.array(test_dataset_label)

idx:  0
idx:  1
idx:  2
idx:  3


In [19]:
import matplotlib.pyplot as plt
import numpy as np
import keras
import keras.backend as K
import tensorflow as tf
from keras.layers import Dropout, Input, Conv2D, MaxPooling2D, Activation, BatchNormalization, Add, Conv2DTranspose, Flatten, Dense, Conv1D, AveragePooling2D, LeakyReLU, PReLU, GlobalAveragePooling2D
from keras.layers import concatenate
from keras.models import Model

import os, pdb, timeit
import numpy as np
from keras.callbacks import TensorBoard
from keras.optimizers import Adam

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import matplotlib.cm as cm

In [20]:
def normalizeDataWholeSeed(data,normalization_type='max'):
    
    if normalization_type == 'max':
        for idx in range(data.shape[0]):
            data[idx,:,:,:] = data[idx,:,:,:]/np.max(abs(data[idx,:,:,:]))
            
    elif normalization_type == 'l2norm':
        from numpy import linalg as LA
        for idx in range(data.shape[0]):
            data[idx,:,:,:] = data[idx,:,:,:]/LA.norm(data[idx,:,:,:]) # L2-norm by default        
        
    return data

In [21]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [22]:
import seaborn as sn
import pandas as pd

In [23]:
def evaluate(model,dataset,dataset_label,normalization_type):
    print("--------------Make Predictions--------------")    
    x = np.array(dataset)
    labels = np.array(dataset_label)
    
    # Normalize the data
    x = normalizeDataWholeSeed(x,normalization_type=normalization_type)
    
    num = x.shape[0]

    print("Testing started")
    tic = timeit.default_timer()
    labels_predicted = model.predict(x)
    toc = timeit.default_timer()
    test_time = toc - tic
    print('Testing time (s) = ' + str(test_time) + '\n')
    
    print("--------")
    # Classification accuracy
    labels_integer_format = labels
    labels_predicted_integer_format = np.argmax(labels_predicted, axis=1)
    
    # Calculate accuracy
    accuracy = accuracy_score(labels_integer_format, labels_predicted_integer_format)
    print("Accuracy:", accuracy)
    
    # Confusion matrices
    confusion_matrix_results = confusion_matrix(labels_integer_format, labels_predicted_integer_format)
    print("Confusion matrix = ")
    print(confusion_matrix_results)
    print("------------------------------------------------")
    
    df_cm = pd.DataFrame(confusion_matrix_results,index =[i for i in VARIETIES],columns=[i for i in VARIETIES])
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16},fmt='.0f') # font size

    plt.show()

    # Calculate precision, recall, and F1-score for each class
    print("Classification Report:")
    print(classification_report(labels_integer_format, labels_predicted_integer_format))

In [24]:
def predict(model,normalization_type):
    evaluate(model,test_dataset,test_dataset_label,normalization_type)

In [25]:
def plot_graph(df,title,xlabel,ylabel,values=['loss'],legends=[]):
    
    for value in values:
        epoch_count = range(1, len(df.index) + 1)
        plt.plot(epoch_count, df[value].tolist())
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    if legends==[]:
        legends = values
    plt.legend(legends, loc='upper left')
    plt.show()

In [26]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K
# Creating Densenet121
def densenet(input_shape, n_classes, filters = 32):
    
    #batch norm + relu + conv
    def bn_rl_conv(x,filters,kernel=1,strides=1):
        
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x
    
    def dense_block(x, repetition):
        
        for _ in range(repetition):
            y = bn_rl_conv(x, 4*filters)
            y = bn_rl_conv(y, filters, 3)
            x = concatenate([y,x])
        return x
        
    def transition_layer(x):
        
        x = bn_rl_conv(x, K.int_shape(x)[-1] //2 )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x
    
    input = Input (input_shape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    
    for repetition in [6,12,24,16]:
        
        d = dense_block(x, repetition)
        x = transition_layer(d)
    x = GlobalAveragePooling2D()(d)
    output = Dense(n_classes, activation = 'softmax')(x)
    
    model = Model(input, output)
    return model

In [27]:
x_training = np.array(x_train)
labels_training = np.array(y_train)

# Normalize the data
x_training = normalizeDataWholeSeed(x_training,normalization_type='max')
x_val_norm = normalizeDataWholeSeed(x_val,normalization_type="max")
test_dataset = normalizeDataWholeSeed(test_dataset,normalization_type='max')
    
# Extract some information
num_training = x_training.shape[0]
N_spatial = x_training.shape[1:3]
N_bands = x_training.shape[3]
batch_size = BATCH_SIZE
num_batch_per_epoch = int(num_training/batch_size)

In [28]:
import pandas as pd
import os

def save_to_csv(file_path, data_frame, header=False):
    file_exists = os.path.exists(file_path)

    if not file_exists or not header:
        data_frame.to_csv(file_path, index=False, mode='w')
    else:
        data_frame.to_csv(file_path, index=False, mode='a', header=False)

In [29]:
VARIETIES

['HD 3086', 'PBW 291', 'DBW 187', 'DBW222']

In [30]:
from tensorflow.keras import datasets, layers, models, losses, Model

In [31]:
model = None

In [32]:
start_epoch = 1

In [33]:
model_name = "DN121_"+"_IC_"+str(TRAIN_IMAGE_COUNT).zfill(5)+"_FilledArea_"+str(FILLED_AREA_RATIO)+"_BandNo_"+str(BAND_NUMBER)+"_ImageHeight_"+str(IMAGE_HEIGHT)+"_ImageWidth_"+str(IMAGE_WIDTH)+"_FILTER_"+str(FILTER)+"_FeatureExtraction_"+str(FEATURE_EXTRACTION)
if REMOVE_NOISY_BANDS:
    model_name+="_REMOVE_NOISY_BANDS_"+str(REMOVE_NOISY_BANDS)+"_NumOfBands_"+str(NUM_OF_BANDS)+"_FB_"+str(FIRST_BAND)+"_LB_"+str(LAST_BAND)
if FILTER == "savgol":
    model_name+="_WINDOW_"+str(WINDOW)+"_ORDER_"+str(ORDER)

if start_epoch != 1:
    model = tf.keras.models.load_model('./DN121models/'+str(start_epoch-1)+model_name)
else:
    input_shape = (30, 30, 168)
    num_classes = 4
    model = densenet(input_shape, num_classes)
    adam_opt = Adam(learning_rate=LEARNING_RATE_BASE, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=adam_opt, metrics=['acc'])

2024-04-08 18:37:30.696435: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14684 MB memory:  -> device: 1, name: Quadro P5000, pci bus id: 0000:9b:00.0, compute capability: 6.1


In [34]:
last_epoch = 100

In [35]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 30, 30, 168)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 15, 15, 64)           526912    ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 8, 8, 64)             0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 batch_normalization (Batch  (None, 8, 8, 64)             256       ['max_pooling2d[0][0]']   

 )                                                                   'concatenate_2[0][0]']       
                                                                                                  
 batch_normalization_8 (Bat  (None, 8, 8, 192)            768       ['concatenate_3[0][0]']       
 chNormalization)                                                                                 
                                                                                                  
 re_lu_8 (ReLU)              (None, 8, 8, 192)            0         ['batch_normalization_8[0][0]'
                                                                    ]                             
                                                                                                  
 conv2d_9 (Conv2D)           (None, 8, 8, 128)            24704     ['re_lu_8[0][0]']             
                                                                                                  
 batch_nor

                                                                                                  
 conv2d_17 (Conv2D)          (None, 4, 4, 32)             36896     ['re_lu_16[0][0]']            
                                                                                                  
 concatenate_7 (Concatenate  (None, 4, 4, 192)            0         ['conv2d_17[0][0]',           
 )                                                                   'concatenate_6[0][0]']       
                                                                                                  
 batch_normalization_17 (Ba  (None, 4, 4, 192)            768       ['concatenate_7[0][0]']       
 tchNormalization)                                                                                
                                                                                                  
 re_lu_17 (ReLU)             (None, 4, 4, 192)            0         ['batch_normalization_17[0][0]
          

 tchNormalization)                                                                                
                                                                                                  
 re_lu_25 (ReLU)             (None, 4, 4, 320)            0         ['batch_normalization_25[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_26 (Conv2D)          (None, 4, 4, 128)            41088     ['re_lu_25[0][0]']            
                                                                                                  
 batch_normalization_26 (Ba  (None, 4, 4, 128)            512       ['conv2d_26[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 re_lu_26 

 batch_normalization_34 (Ba  (None, 4, 4, 128)            512       ['conv2d_34[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 re_lu_34 (ReLU)             (None, 4, 4, 128)            0         ['batch_normalization_34[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_35 (Conv2D)          (None, 4, 4, 32)             36896     ['re_lu_34[0][0]']            
                                                                                                  
 concatenate_16 (Concatenat  (None, 4, 4, 480)            0         ['conv2d_35[0][0]',           
 e)                                                                  'concatenate_15[0][0]']      
          

...

                                                                                                  
 conv2d_78 (Conv2D)          (None, 2, 2, 32)             36896     ['re_lu_77[0][0]']            
                                                                                                  
 concatenate_37 (Concatenat  (None, 2, 2, 896)            0         ['conv2d_78[0][0]',           
 e)                                                                  'concatenate_36[0][0]']      
                                                                                                  
 batch_normalization_78 (Ba  (None, 2, 2, 896)            3584      ['concatenate_37[0][0]']      
 tchNormalization)                                                                                
                                                                                                  
 re_lu_78 (ReLU)             (None, 2, 2, 896)            0         ['batch_normalization_78[0][0]
          

 tchNormalization)                                                                                
                                                                                                  
 re_lu_86 (ReLU)             (None, 2, 2, 1024)           0         ['batch_normalization_86[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_87 (Conv2D)          (None, 2, 2, 512)            524800    ['re_lu_86[0][0]']            
                                                                                                  
 average_pooling2d_2 (Avera  (None, 1, 1, 512)            0         ['conv2d_87[0][0]']           
 gePooling2D)                                                                                     
                                                                                                  
 batch_nor

 concatenate_45 (Concatenat  (None, 1, 1, 640)            0         ['conv2d_95[0][0]',           
 e)                                                                  'concatenate_44[0][0]']      
                                                                                                  
 batch_normalization_95 (Ba  (None, 1, 1, 640)            2560      ['concatenate_45[0][0]']      
 tchNormalization)                                                                                
                                                                                                  
 re_lu_95 (ReLU)             (None, 1, 1, 640)            0         ['batch_normalization_95[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_96 (Conv2D)          (None, 1, 1, 128)            82048     ['re_lu_95[0][0]']            
          

                                                                    ]']                           
                                                                                                  
 conv2d_104 (Conv2D)         (None, 1, 1, 128)            98432     ['re_lu_103[0][0]']           
                                                                                                  
 batch_normalization_104 (B  (None, 1, 1, 128)            512       ['conv2d_104[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 re_lu_104 (ReLU)            (None, 1, 1, 128)            0         ['batch_normalization_104[0][0
                                                                    ]']                           
                                                                                                  
 conv2d_10

 re_lu_112 (ReLU)            (None, 1, 1, 128)            0         ['batch_normalization_112[0][0
                                                                    ]']                           
                                                                                                  
 conv2d_113 (Conv2D)         (None, 1, 1, 32)             36896     ['re_lu_112[0][0]']           
                                                                                                  
 concatenate_54 (Concatenat  (None, 1, 1, 928)            0         ['conv2d_113[0][0]',          
 e)                                                                  'concatenate_53[0][0]']      
                                                                                                  
 batch_normalization_113 (B  (None, 1, 1, 928)            3712      ['concatenate_54[0][0]']      
 atchNormalization)                                                                               
          

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

tic = start_timer()
while start_epoch<=last_epoch:
    print("\nEpoch: ",start_epoch)
    history = model.fit(x_training, labels_training, batch_size=batch_size, epochs = 1, validation_data=(x_val_norm, y_val), verbose=2)
    model.save('./DN121models/'+str(start_epoch)+model_name)
    print("Model saved on epoch: ",start_epoch)
    
    history_dataframe = pd.DataFrame.from_dict(history.history)
    save_to_csv('./csvs/'+model_name+'.csv', history_dataframe, header=True)
    print("added to csv")
    
    y_pred = model.predict(test_dataset)

    y_pred_label = np.argmax(y_pred, axis=1)

    # Calculate accuracy
    accuracy = accuracy_score(test_dataset_label, y_pred_label)
    print("Accuracy:", accuracy)

    # Calculate confusion matrix
    cm = confusion_matrix(test_dataset_label, y_pred_label)
    print("Confusion Matrix:")
    print(cm)

    # Calculate precision, recall, and F1-score for each class
    print("Classification Report:")
    print(classification_report(test_dataset_label, y_pred_label))
    start_epoch+=1
    
toc = end_timer()
show_time(tic,toc)


Testing started

Epoch:  1


2024-04-08 18:39:09.397190: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8907
2024-04-08 18:39:12.355362: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x77d3b2a4b090 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-08 18:39:12.355433: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro P5000, Compute Capability 6.1
2024-04-08 18:39:14.075975: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-08 18:39:20.661838: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


600/600 - 273s - loss: 1.1006 - acc: 0.5073 - val_loss: 2.6301 - val_acc: 0.2781 - 273s/epoch - 455ms/step
INFO:tensorflow:Assets written to: ./DN121models/1DN121__IC_01200_FilledArea_0.9_BandNo_60_ImageHeight_30_ImageWidth_30_FILTER_msc_FeatureExtraction_none/assets


INFO:tensorflow:Assets written to: ./DN121models/1DN121__IC_01200_FilledArea_0.9_BandNo_60_ImageHeight_30_ImageWidth_30_FILTER_msc_FeatureExtraction_none/assets


Model saved on epoch:  1
added to csv
50/50 [==============================] - 8s 33ms/step
Accuracy: 0.285625
Confusion Matrix:
[[ 28   0 370   2]
 [  1  23 376   0]
 [  0   3 397   0]
 [  3   0 388   9]]
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.07      0.13       400
           1       0.88      0.06      0.11       400
           2       0.26      0.99      0.41       400
           3       0.82      0.02      0.04       400

    accuracy                           0.29      1600
   macro avg       0.71      0.29      0.17      1600
weighted avg       0.71      0.29      0.17      1600


Epoch:  2
600/600 - 91s - loss: 0.7968 - acc: 0.6715 - val_loss: 1.1315 - val_acc: 0.5500 - 91s/epoch - 152ms/step
INFO:tensorflow:Assets written to: ./DN121models/2DN121__IC_01200_FilledArea_0.9_BandNo_60_ImageHeight_30_ImageWidth_30_FILTER_msc_FeatureExtraction_none/assets


...

Model saved on epoch:  53
added to csv
50/50 [==============================] - 2s 31ms/step
Accuracy: 0.86875
Confusion Matrix:
[[247  35 118   0]
 [  9 375  16   0]
 [ 10  22 368   0]
 [  0   0   0 400]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.62      0.74       400
           1       0.87      0.94      0.90       400
           2       0.73      0.92      0.82       400
           3       1.00      1.00      1.00       400

    accuracy                           0.87      1600
   macro avg       0.88      0.87      0.86      1600
weighted avg       0.88      0.87      0.86      1600


Epoch:  54
600/600 - 91s - loss: 2.2558e-06 - acc: 1.0000 - val_loss: 0.6656 - val_acc: 0.9175 - 91s/epoch - 152ms/step
INFO:tensorflow:Assets written to: ./DN121models/54DN121__IC_01200_FilledArea_0.9_BandNo_60_ImageHeight_30_ImageWidth_30_FILTER_msc_FeatureExtraction_none/assets


INFO:tensorflow:Assets written to: ./DN121models/54DN121__IC_01200_FilledArea_0.9_BandNo_60_ImageHeight_30_ImageWidth_30_FILTER_msc_FeatureExtraction_none/assets


Model saved on epoch:  54
added to csv
50/50 [==============================] - 2s 31ms/step
Accuracy: 0.869375
Confusion Matrix:
[[246  35 119   0]
 [  8 376  16   0]
 [ 10  21 369   0]
 [  0   0   0 400]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.61      0.74       400
           1       0.87      0.94      0.90       400
           2       0.73      0.92      0.82       400
           3       1.00      1.00      1.00       400

    accuracy                           0.87      1600
   macro avg       0.88      0.87      0.87      1600
weighted avg       0.88      0.87      0.87      1600


Epoch:  55
600/600 - 91s - loss: 1.6714e-06 - acc: 1.0000 - val_loss: 0.6770 - val_acc: 0.9181 - 91s/epoch - 152ms/step
INFO:tensorflow:Assets written to: ./DN121models/55DN121__IC_01200_FilledArea_0.9_BandNo_60_ImageHeight_30_ImageWidth_30_FILTER_msc_FeatureExtraction_none/assets


INFO:tensorflow:Assets written to: ./DN121models/55DN121__IC_01200_FilledArea_0.9_BandNo_60_ImageHeight_30_ImageWidth_30_FILTER_msc_FeatureExtraction_none/assets


Model saved on epoch:  55
added to csv
50/50 [==============================] - 2s 30ms/step
Accuracy: 0.87
Confusion Matrix:
[[245  36 119   0]
 [  8 376  16   0]
 [ 10  19 371   0]
 [  0   0   0 400]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.61      0.74       400
           1       0.87      0.94      0.90       400
           2       0.73      0.93      0.82       400
           3       1.00      1.00      1.00       400

    accuracy                           0.87      1600
   macro avg       0.88      0.87      0.87      1600
weighted avg       0.88      0.87      0.87      1600


Epoch:  56


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(test_dataset)

y_pred_label = np.argmax(y_pred, axis=1)

# Calculate accuracy
accuracy = accuracy_score(test_dataset_label, y_pred_label)
print("Accuracy:", accuracy)

# Calculate confusion matrix
cm = confusion_matrix(test_dataset_label, y_pred_label)
print("Confusion Matrix:")
print(cm)

# Calculate precision, recall, and F1-score for each class
print("Classification Report:")
print(classification_report(test_dataset_label, y_pred_label))

In [ ]:
df_cm = pd.DataFrame(cm,index =[i for i in VARIETIES],columns=[i for i in VARIETIES])
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16},fmt='.0f') # font size

plt.show()